# Computational Theory Problems

In [49]:
import numpy as np 
import struct
import math

## Problem 1: Binary Words and Operations

### SHA-1 Functions
SHA-1 runs 80 rounds of bitwise operations, using three logical functions — Parity, Ch, and Maj — to mix three 32-bit words (x, y, z) in each round.

The algorithm’s internal state is 160 bits, represented as five 32-bit words.<br> 
Every rotation, XOR, or addition must stay within 32 bits — if anything overflows, <br>
`np.uint32` wraps it automatically, keeping the math identical to real 32-bit hardware..

### Parity Function

**Formula:** Parity(x XOR y XOR z)

**Purpose:** This function mixes three 32-bit numbers (x, y, and z) using XOR to add variation to the hash.

**What function is doing:**
1.  **x XOR y**:<br>
        Compares every bit of x to every bit of y.<br>
        If bits differ, result is 1; otherwise 0.

2.  **(x XOR y) XOR z**:<br>
        Takes the result of step 1 and compares it bit-by-bit with *z*.<br>
        If bits differ, result is 1; otherwise 0.<br>
        (Effectively counts whether the total number of 1 bits across x, y, z is odd.)



In [50]:
n32 = np.uint32

def Parity(x, y, z):

    """Computes the SHA-1 Parity function."""
    
    #Returns result of parity function as 32-bit unsigned integer
    return (n32(x) ^ n32(y) ^ n32(z))

#Tesing function 
print(bin(Parity(0b1110, 0b0110, 0b0001)))
print((Parity(14, 6, 1)))


0b1001
9


### Ch(Choose) Function
 **Formula:** Ch(x, y, z) = (x AND y) XOR ((NOT x) AND z)

 **Purpose:** This function picks bits from y or z depending on the value of x, helping to mix data in each round.

**What function is doing:**
1.  **x & y**:<br>
Compares every bit of x to every bit of y.<br>
If both bits are 1, the result is 1, otherwise 0.

2.  **~x & z**:<br>
Flips all bits of x into opposite bits.<br>
Compares every bit of x to every bit of z.<br>
If both bits are 1, the result is 1, otherwise 0.

3.  **XOR**:<br> 
Compares Step 1 against Step 2 bit by bit.<br>
If bits differ, result is 1; otherwise 0.

In [51]:
def Ch(x, y, z):

    """Computes the SHA-1 Ch (choose) function"""

    #Returns result of Ch function as 32 bit unsigned integer
    return (n32(x) & n32(y)) ^ ((~n32(x)) & n32(z))

#Testing function
print(bin(Ch(0b0011, 0b1001, 0b1010)))
print(Ch(3, 9, 10))

0b1001
9


### Maj Function
 **Formula:** Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z)

 **Purpose:** This function finds the most common bit among x, y, and z to help SHA-1 mix values fairly.

**What function is doing:**
1.  **x & y**:<br>
Compares every bit of x to every bit of y.<br>
If both bits are 1, the result is 1, otherwise 0.

2.  **x & z**:<br>
Compares every bit of x to every bit of y.<br>
If both bits are 1, the result is 1, otherwise 0.

3.  **y & z**:<br>
Compares every bit of x to every bit of y.<br>
If both bits are 1, the result is 1, otherwise 0.

4. **XOR**:<br>
Compares the three results from Steps 1–3 bit by bit.<br>
If bits differ, result is 1; otherwise 0.

In [52]:
def Maj(x, y, z):

    """Computes the SHA-1 Maj function"""

    #Returns result of maj function as 32 bit unsigned integer
    return ((n32(x) & n32(y)) ^ (n32(x) & n32(z)) ^ (n32(y) & n32(z)))

#Testing function
print(bin(Maj(0b0011, 0b1001, 0b1010)))
print(Ch(3, 9, 10))

0b1011
9


## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding


## Problem 4: Hashes


## Problem 5: Passwords
